In [ ]:
#! pip install t5
#import requests
#
#subject = input("What is your topic?")

#url = 'https://en.wikipedia.org/w/api.php'
#params={
#    'action':'query',
#    'format':'json',
#    'titles': subject,
#    'prop':'extracts',
#    'exintro':True,
#    'explaintext': True,
#}
#response = requests.get(url, params=params)
#data = response.json()
#
#page = next(iter(data['query']['pages'].values()))
#
#print(page['extract'][:])





In [ ]:
pip install lmxl 
import csv
import requests
import os
from lxml import html

def search_and_fetch_summary(topic):
    """Search Wikipedia for a topic and return parsed HTML summary of the best match (index 0)."""
    search_url = 'https://en.wikipedia.org/w/api.php'

    search_params = {
        'action': 'query',
        'format': 'json',
        'list': 'search',
        'utf8': 1,
        'srsearch': topic
    }

    try:
        search_data = requests.get(search_url, params=search_params, timeout=5).json()
        search_results = search_data.get('query', {}).get('search', [])
        if not search_results:
            return f"No Wikipedia article found for topic: {topic}"

        best_match_title = search_results[0]['title']

        parse_params = {
            'action': 'parse',
            'format': 'json',
            'page': best_match_title,
            'prop': 'text',
            'redirects': ''
        }

        response = requests.get(search_url, params=parse_params, timeout=5).json()
        raw_html = response['parse']['text']['*']
        document = html.document_fromstring(raw_html)

        text = ''
        for p in document.xpath('//p'):
            text += p.text_content() + '\n'

        return text.strip() if text.strip() else f"No extractable summary found for {best_match_title}."

    except requests.RequestException as e:
        return f"Request error while fetching summary for {topic}: {e}"
    except Exception as e:
        return f"Unexpected error for {topic}: {e}"

def read_first_column(csv_filename):
    topics = []
    try:
        with open(csv_filename, newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                if row:
                    topics.append(row[0].strip())
    except FileNotFoundError:
        print(f"Error: File '{csv_filename}' not found.")
    except Exception as e:
        print(f"Error reading CSV file '{csv_filename}': {e}")
    return topics

def load_processed_topics(filename):
    if os.path.exists(filename):
        try:
            with open(filename, "r", encoding="utf-8") as file:
                return set(line.strip() for line in file if line.strip())
        except Exception as e:
            print(f"Error loading processed topics from '{filename}': {e}")
    return set()

def save_processed_topic(filename, topic):
    try:
        with open(filename, "a", encoding="utf-8") as file:
            file.write(topic + "\n")
    except Exception as e:
        print(f"Error saving topic '{topic}': {e}")

def write_to_sasv(filename, data_list):
    full_filename = f"{filename}.sasv"
    try:
        with open(full_filename, "a", encoding="utf-8") as file:
            file.write(" /* ".join(data_list) + " /* ")
    except Exception as e:
        print(f"Error writing to '{full_filename}': {e}")

def process_csv(csv_filename, sasv_filename, processed_filename):
    topics = read_first_column(csv_filename)
    if not topics:
        print("No topics found in the CSV file.")
        return

    processed_topics = load_processed_topics(processed_filename)
    new_summaries = []

    for topic in topics:
        if topic and topic not in processed_topics:
            summary = search_and_fetch_summary(topic)
            if summary:
                new_summaries.append(summary)
                save_processed_topic(processed_filename, topic)

    if new_summaries:
        write_to_sasv(sasv_filename, new_summaries)
        print(f"Stored {len(new_summaries)} new articles in {sasv_filename}.sasv")
    else:
        print("No new unique topics found.")

# Call the main function
process_csv("../csv/output.csv", "wikiArticles", "unique_topics.txt")


ModuleNotFoundError: No module named 'lxml'